In [ ]:
import tensorflow as tf
DEVICE = 'GPU'
numWorkers = len(tf.config.list_physical_devices('GPU'))
if numWorkers == 0:
    DEVICE = 'CPU'
    numWorkers = !cat /proc/cpuinfo | grep processor | wc -l
    numWorkers = int(numWorkers[0])
    
print(numWorkers)

In [ ]:
import pandas
import numpy as np
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from multiprocessing.pool import ThreadPool
import talos
import os
import math
from matplotlib import pyplot
import tensorflow_datasets as tfds
from pathlib import Path
from copy import deepcopy
from dataset_loader import load_tf_dataset
from models_keras import get_model
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#global parameters for grid search over activation function
degree = 3
interval = 0

def reluApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.7146 + 1.5000*K.pow(x/interval,1)+0.8793*K.pow(x/interval,2)
        if interval == 5:
            return 0.7865 + 2.5000*K.pow(x/interval,1)+1.88*K.pow(x/interval,2)
        if interval == 7:
            return 0.9003 + 3.5000*K.pow(x/interval,1)+2.9013*K.pow(x/interval,2)
        if interval == 10:
            return 1.1155 + 5*K.pow(x/interval,1)+4.4003*K.pow(x/interval,2)
        if interval == 12:
            return 1.2751 + 6*K.pow(x/interval,1)+5.3803*K.pow(x/interval,2)
    if degree == 5:  
        if interval == 7:
            return 0.7521 + 3.5000*K.pow(x/interval,1)+4.3825*K.pow(x/interval,2)-1.7281*K.pow(x/interval,4)
        if interval == 20:
            return 1.3127 + 10*K.pow(x/interval,1)+15.7631*K.pow(x/interval,2)-7.6296*K.pow(x/interval,4)
def sigmoidApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.5 + 0.6997*K.pow(x/interval,1)-0.2649*K.pow(x/interval,3)
        if interval == 5:
            return 0.5 + 0.9917*K.pow(x/interval,1)-0.5592*K.pow(x/interval,3)
        if interval == 7:
            return 0.5 + 1.1511*K.pow(x/interval,1)-0.7517*K.pow(x/interval,3)
        if interval == 8:
            return 0.5 + 1.2010*K.pow(x/interval,1)-0.8156*K.pow(x/interval,2)
        if interval == 12:
            return 0.5 + 1.2384*K.pow(x/interval,1)-0.8647*K.pow(x/interval,2)
def tanApprox(x):
    if degree == 3: 
        if interval == 1:
            return 0.9797*K.pow(x/interval,1)-0.2268*K.pow(x/interval,3)        
        if interval == 2:
            return 1.7329*K.pow(x/interval,1)-0.8454*K.pow(x/interval,3)
        if interval == 3:
            return 2.1673*K.pow(x/interval,1)-1.3358*K.pow(x/interval,3)
        if interval == 5:
            return 2.5338*K.pow(x/interval,1)-1.8051*K.pow(x/interval,3)
        if interval == 7:
            return 2.6629*K.pow(x/interval,1) -1.9801*K.pow(x/interval,3)
        if interval == 12:
            return 2.7599*K.pow(x/interval,1)-2.1140*K.pow(x/interval,2)
    if degree == 12:
        print('ooopssss')

In [ ]:
def randomSplitClientsData(data, labels, ds_info):
    
    numParties = ds_info['num_clients']
    sample_height, sample_width, sample_channels = ds_info['sample_shape']
    num_classes = ds_info['num_classes']
    
    numSamplesPerClient = int(data.shape[0]/numParties)
    #print(numSamplesPerClient)
    clientsData = np.zeros((numParties,int(numSamplesPerClient),sample_height,sample_width,sample_channels))
    clientsDataLabels = np.zeros((numParties,int(numSamplesPerClient),num_classes))
    #print(numSamplesPerClient)
    ind = 0
    for i in range(numParties):
        clientsData[i] = data[ind:ind+numSamplesPerClient]
        clientsDataLabels[i]=labels[ind:ind+numSamplesPerClient]
        ind = ind+numSamplesPerClient
    return clientsData, clientsDataLabels

def prepare_data_for_X_clients(x_train, y_train, ds_info):
    clientsData, clientsDataLabels = randomSplitClientsData(x_train, y_train, ds_info)
    return clientsData, clientsDataLabels

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history, params):
    print('##########################################################')
    print(params)
    # plot loss
    pyplot.subplot(211)
    pyplot.title('MSE')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #pyplot.savefig(filename + '_plot.png')
    #pyplot.close()
    pyplot.show()
    print('##########################################################')

In [ ]:
def experiment(x_train, y_train, x_val, y_val, params):
    
    optimizer = SGD(learning_rate=params['learn_rate'], momentum=params['momentum'], nesterov=False, name='SGD')
    
    model = get_model(params, ds_info)
    model.compile(optimizer=optimizer,
                  loss="mean_squared_error",
                  metrics=['accuracy', tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

    history = model.fit(x=x_train,
                    y=y_train,
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=(x_val, y_val),
                    verbose=0)

    hist.append(history)
    hist_params.append(params)
        
    return history, model

In [ ]:
def client_gridsearch(work):
    
    client_number, clientData, clientDataLabels, param_grid = work
    
    free = np.where(workers == 1)
    i = free[0][0]
    workers[i] = 0

    #Distribute load accross DEVICEs
    with tf.device(f"/{DEVICE}:{i}"):
        print(f"training on {DEVICE}: {i}, interval is {interval}")
        try:
            scan_results = talos.Scan(x=clientData,
                                      y=clientDataLabels,
                                      params=param_grid,
                                      model=experiment,
                                      experiment_name=f"{experiment_name}_{client_number}")
            scan_res[client_number]=scan_results
            
        except:
            pass
            
        print(f"client running on {DEVICE}: {i} finished")

        workers[i] = 1

        return


def grid_search_for_X_clients(num_clients, clientsData, clientsDataLabels, param_grid):
    
    global scan_res
    global workers
    global hist
    global hist_params
    
    scan_res = np.zeros(num_clients, dtype=object)
    workers = np.ones(numWorkers)
    
    hist = []
    hist_params = []
                
    work = [(i, clientsData[i], clientsDataLabels[i], deepcopy(param_grid)) for i in range(num_clients)]
    
    with ThreadPool(len(workers)) as p:
        p.map(client_gridsearch, work)

    return scan_res

In [ ]:
def client_test_metrics(work):
    
        client_number, clientData, clientDataLabels, avg_test_params = work
    
        free = np.where(workers == 1)
        i = free[0][0]
        workers[i] = 0
    
        #Distribute load accross DEVICEs
        with tf.device(f"/{DEVICE}:{i%numWorkers}"):
            
            print(f"training on {DEVICE}: {i}, interval is {interval}")
            model = get_model(avg_test_params, ds_info)
            optimizer = SGD(learning_rate=avg_test_params['learn_rate'], momentum=avg_test_params['momentum'], nesterov=False, name='SGD')
            model.compile(optimizer=optimizer, loss="mean_squared_error",
                          metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
                          run_eagerly=False)

            try:
                model.fit(x=clientData,
                            y=clientDataLabels,
                            epochs=avg_test_params['epochs'],
                            batch_size=avg_test_params['batch_size'],
                            verbose=0)

                print(f"client running on {DEVICE}: {i} finished")
                metrics = model.evaluate(x_test, y_test)
                metrics_res[client_number] = metrics
                
            except:
                pass
            
            workers[i] = 1
            
        return

def test_metrics_for_X_clients(num_clients, clientsData, clientsDataLabels, avg_test_params):

    global metrics_res
    global workers
    
    metrics_res = np.zeros(num_clients, dtype=object)   
    workers = np.ones(numWorkers)
                
    work = [(i, clientsData[i], clientsDataLabels[i], avg_test_params) for i in range(num_clients)]
    
    with ThreadPool(len(workers)) as p:
        p.map(client_test_metrics, work)
    
    return metrics_res

In [ ]:
def run(interval_params, ds, test_split, ds_info):
    num_clients = ds_info['num_clients']
    
    (x_train, y_train) = ds
    
    global x_test
    global y_test
    (x_test, y_test) = test_split
    
    Path(experiment_name+"_res/intervals_res"+str(num_clients)).mkdir(parents=True, exist_ok=True)
    
    interval_res = list()
    clientsData, clientsDataLabels = prepare_data_for_X_clients(x_train, y_train, ds_info)
    
    # Proceed to interval search

    for i in intervals:
        global interval
        interval = i

        interval_res.append(grid_search_for_X_clients(num_clients, clientsData, clientsDataLabels, intervals_params))

    # Extract best interval from search
    
    best_interval = 0

    choice_ratio = np.zeros(num_clients, dtype=object)
    choice_ratio.fill((0,0.0))

    for index, i_res in enumerate(interval_res):

        i_res = [r for r in i_res if not r == 0]

        for c, client_scan in enumerate(i_res):
            if not client_scan == 0:
                curr_acc = client_scan.data.head(1)['val_accuracy'].item()

                display(client_scan.data)

                if choice_ratio[c][1] <= curr_acc:
                    choice_ratio[c] = (intervals[index], curr_acc)      

                ## Write dataframes to files
                client_scan.data.to_csv(experiment_name+"_res/intervals_res"+str(num_clients)+"/intervals_res_client"+str(c)+"_interval_"+str(intervals[index])+".csv")

    choice_ratio = np.array([e[0] for e in choice_ratio])

    res_ratio = np.zeros(len(intervals), dtype=float)

    for i,e in enumerate(intervals):
        res_ratio[i] = np.count_nonzero(choice_ratio == e)

    res_ratio = res_ratio / num_clients

    best_interval = intervals[np.argmax(res_ratio)]
    print("Ratios:", res_ratio, "Best interval:", best_interval)

    ####### Retrain each client to run the test set and get best metrics
    intervals_test_res = []
    interval = best_interval
    intervals_params_test = dict(learn_rate=intervals_params['learn_rate'][0],
                            batch_size=intervals_params['batch_size'][0],
                            momentum=intervals_params['momentum'][0],
                            epochs=intervals_params['epochs'][0], 
                            act_fn=intervals_params['act_fn'][0])

    clientsData, clientsDataLabels = prepare_data_for_X_clients(x_train, y_train, ds_info)

    intervals_test_res = test_metrics_for_X_clients(num_clients, clientsData, clientsDataLabels, intervals_params_test)

    best_index = 1
    intervals_test_res = [r for r in intervals_test_res if not r == 0]
    for index, i_res in enumerate(intervals_test_res):

        if i_res[1] >= intervals_test_res[best_index][1]:
            best_index = index

    print("Best test accuracy :", intervals_test_res[best_index][1])
    print("Loss :", intervals_test_res[best_index][0])
    print("Precision :", intervals_test_res[best_index][2])
    print("Recall :", intervals_test_res[best_index][3])

In [ ]:
dataset_name = 'svhn_cropped'
experiment_name = f"{dataset_name}_iid"

# Set approximated activation function and best parameters
approx_act_fn = [reluApprox]
# Set intervals to search, check intervals in the approx function
intervals = [3,5,7,10,12]

intervals_params = dict(learn_rate=[0.193],
                        batch_size=[8],
                        momentum=[0.776],
                        epochs=[29], 
                        act_fn=approx_act_fn)

ds, test_split, ds_info = load_tf_dataset(dataset_name=dataset_name, decentralized=True, display=True)
ds_info['num_clients'] = 10
run(intervals_params, ds, test_split, ds_info)